In [1]:
import torch
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from qiskit.tools.monitor import job_monitor
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit import Aer, execute, IBMQ
from qiskit.extensions import XGate, UnitaryGate

import sys
import os
import time
import functools
import IPython.display

from lib.utils import *
from lib.U_layer import *
from lib.P_layer import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

print = functools.partial(print, flush=True)

In [2]:
# account = IBMQ.load_account()
# provider = IBMQ.get_provider(hub='ibm-q-research', group='uni-cali-la-1', project='main')
# provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-research', group='uni-cali-la-1', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q-research', group='uni-cali-la-1', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q-research', group='uni-cali-la-1', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q-research', group='uni-cali-la-1', project='main')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q-research', group='uni-cali-la-1', project='main')>,
 <IBMQBackend('ibmq_casablanca') from IBMQ(hub='ibm-q-research', group='uni-cali-la-1', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-research', group='uni-cali-la-1', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-research', group='uni-cali-la-1', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-research', group='uni-cali-la-1', project='main')>,
 <IBMQBackend('ibmq_jakarta') from IBMQ(hub='ibm-q-research', group='uni-ca

# Initialization

In [3]:
interest_num = [3,6]
ori_img_size = 28
img_size = 4
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 1
inference_batch_size = 1
data_path = './data'

qc_shots = 8192


# convert data to torch.FloatTensor
transform = transforms.Compose([transforms.Resize((ori_img_size,ori_img_size)),
                                transforms.ToTensor()])
# Path to MNIST Dataset
train_data = datasets.MNIST(root=data_path, train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root=data_path, train=False,
                                  download=True, transform=transform)

train_data = select_num(train_data,interest_num)
test_data =  select_num(test_data,interest_num)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=num_workers, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=inference_batch_size, 
    num_workers=num_workers, shuffle=True, drop_last=True)



# Model initialization
weight_1_1 = torch.tensor([1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,    1.,  1.])
weight_1_2 = torch.tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1., -1.,-1., -1.])

weight_2_1 = torch.tensor([1.,  -1.])
norm_flag_1 = True
norm_para_1 = torch.tensor(0.3060)

weight_2_2 = torch.tensor([-1.,  -1.])
norm_flag_2 = False
norm_para_2 = torch.tensor(0.6940)

weights = [weight_1_1, weight_1_2, weight_2_1, weight_2_2]
flags = [norm_flag_1, norm_flag_2]
params = [norm_para_1, norm_para_2]

weights_1 = [weight_1_1, weight_2_1]
weights_2 = [weight_1_2, weight_2_2]

# Multi-Class

In [ ]:
max_itr = 2
saving = False
quiet = True
simulation = True

p_pred = []
u_pred = []
p_prob = []
u_prob = []
itr = 0

for batch_idx, (data, target) in enumerate(test_loader):

    torch.set_printoptions(threshold=sys.maxsize)
    # print("Batch Id: {}, Target: {}".format(batch_idx,target))
    quantum_matrix, qantum_data = data_pre_pro(
        torchvision.utils.make_grid(data), img_size, verbose=False)

    p_circ = p_circ_gen(quantum_matrix, weights, flags, params)
    u_circ = u_circ_gen(quantum_matrix, weights, flags, params)

    # p_circ
    counts = fire_ibmq(p_circ, qc_shots, Simulation=simulation, quiet=quiet)
    (mycount, bits) = analyze(counts)
    class_prob = []
    for b in range(bits):
        class_prob.append(float(mycount[b])/qc_shots)
    
    p_prob.append(class_prob)
    result = abs((class_prob.index(max(class_prob)) - target[0]).numpy()) # 0 if correct, 1 if not
    p_pred.append(result)

    if quiet==False:
        print("="*10, "Non-Optimized Circuit", "="*10)
        print("Non-Optimized Circuit Depth:", p_circ.depth())
        print("Result of non-optimized QC:", class_prob)
        print("Prediction class: {}".format(class_prob.index(max(class_prob))))
        print("Target class: {}".format(target[0]))

        if class_prob.index(max(class_prob)) == target[0]:
            print("Correct prediction")
        else:
            print("Incorrect prediction")

        print("="*30)

    # u_circ
    opt_counts = fire_ibmq(u_circ, qc_shots, Simulation=simulation, quiet=quiet)
    (opt_mycount, bits) = analyze(opt_counts)
    opt_class_prob = []
    for b in range(bits):
        opt_class_prob.append(float(opt_mycount[b])/qc_shots)

    u_prob.append(opt_class_prob)
    result = abs((opt_class_prob.index(max(opt_class_prob)) - target[0]).numpy())
    u_pred.append(result)

    if quiet==False:
        print("="*10, "Optimized Circuit", "="*10)
        print("Optimized Circuit Depth:", u_circ.depth())
        print("Result of optimized QC:", opt_class_prob)
        print("Prediction class: {}".format(opt_class_prob.index(max(opt_class_prob))))
        print("Target class: {}".format(target[0]))

        if opt_class_prob.index(max(opt_class_prob)) == target[0]:
            print("Correct prediction")
        else:
            print("Incorrect prediction")

        print("="*30)
    
    itr += 1
    IPython.display.clear_output(wait=True)
    print("iterantion", itr, "out of", min(max_itr, len(test_loader)))
    if itr >= max_itr:
        break

if saving:
    t = time.localtime()
    dir = "./results/multi_"+time.strftime('%m-%d_%H-%M', t)

    if not os.path.exists(dir):
        os.makedirs(dir)

    df_pred = pd.DataFrame([p_pred, u_pred])
    df_pred.to_csv(dir+"/predictions.csv", index=False)

    dfp = pd.DataFrame(p_prob, columns=["p[0]", "p[1]"])
    dfu = pd.DataFrame(u_prob, columns=["u[0]", "u[1]"])
    df_prob = pd.concat([dfp, dfu], axis=1)
    df_pred.to_csv(dir+"/probilities.csv", index=False)

## Analysis

In [6]:
data_path = ".\\results\\multi_4x4_full_set"

df_pred = pd.read_csv(data_path + '\\predictions.csv')
p_pred = df_pred.iloc[0, :].to_numpy()
u_pred = df_pred.iloc[1, :].to_numpy()

df_prob = pd.read_csv(data_path + '\\probabilities.csv')
p_prob = df_prob.iloc[:, 0:2].to_numpy()
u_prob = df_prob.iloc[:, 2:4].to_numpy()

p_acc = 1 - p_pred.sum()/p_pred.shape[0]
u_acc = 1 - u_pred.sum()/u_pred.shape[0]

print("p_acc:", p_acc, '\n' "u_acc:", u_acc)

p_acc: 0.9634146341463414 
u_acc: 0.9649390243902439


# Single Class

In [7]:
max_itr = 1000
saving = False
quiet = True
simulation = True
backend = 'ibmq_casablanca'

p_pred = []
u_pred = []
p_prob = []
u_prob = []
itr = 0

for batch_idx, (data, target) in enumerate(test_loader):

    torch.set_printoptions(threshold=sys.maxsize)
    # print("Batch Id: {}, Target: {}".format(batch_idx,target))
    quantum_matrix, qantum_data = data_pre_pro(
        torchvision.utils.make_grid(data), img_size, verbose=False)

    p_single_circ = p_single_circ_gen(quantum_matrix, weights_1, norm_flag_1, norm_para_1)
    counts = fire_ibmq(p_single_circ, qc_shots, Simulation=simulation, backend_name=backend, quiet=quiet)
    
    (mycount, bits) = analyze(counts)
    class_prob = []
    for b in range(bits):
        class_prob.append(float(mycount[b])/qc_shots)

    class_prob.append(1-class_prob[0])
    result = abs((class_prob.index(max(class_prob)) - target[0]).numpy()) # 0 if correct, 1 if not

    p_prob.append(class_prob)
    p_pred.append(result)

    # print(result==0, class_prob[0])
    
    itr += 1
    IPython.display.clear_output(wait=True)
    print("iterantion", itr, "out of", min(max_itr, len(test_loader)))
    if itr >= max_itr:
        break

if saving:
    t = time.localtime()
    dir = "./results/single_"+time.strftime('%m-%d_%H-%M', t)

    if not os.path.exists(dir):
        os.makedirs(dir)

    # df_pred = pd.DataFrame([p_pred, u_pred])
    df_pred = pd.DataFrame(p_pred)
    df_pred.to_csv(dir+"/predictions.csv", index=False)

    dfp = pd.DataFrame(p_prob, columns=["p[0]", "p[1]"])
    # dfu = pd.DataFrame(u_prob, columns=["u[0]", "u[1]"])
    dfp.to_csv(dir+"/probilities.csv", index=False)



iterantion 7 out of 1000
Job Status: job is being initialized

In [30]:
# singular
data_path = "results\\07-28_18-42_single"

df_pred = pd.read_csv(data_path + '\\predictions.csv')
p_pred = df_pred.iloc[:, 0].to_numpy()

df_prob = pd.read_csv(data_path + "\\probilities.csv")
p_prob = df_prob.iloc[:, 0:2].to_numpy()

p_acc = 1 - p_pred.sum()/p_pred.shape[0]
print(p_acc)

0.76


# Dual Class

In [8]:
max_itr = 2000
saving = True
quiet = True
simulation = True

p_pred = []
u_pred = []
p_prob = []
u_prob = []
itr = 0

for batch_idx, (data, target) in enumerate(test_loader):

    torch.set_printoptions(threshold=sys.maxsize)
    # print("Batch Id: {}, Target: {}".format(batch_idx,target))
    quantum_matrix, qantum_data = data_pre_pro(
        torchvision.utils.make_grid(data), img_size, verbose=False)

    p_single_circ_1 = p_single_circ_gen(quantum_matrix, weights_1, norm_flag_1, norm_para_1)
    p_single_circ_2 = p_single_circ_gen(quantum_matrix, weights_2, norm_flag_2, norm_para_2)
    class_prob = []
    
    counts = fire_ibmq(p_single_circ_1, qc_shots, Simulation=simulation, quiet=quiet)
    (mycount, bits) = analyze(counts)
    for b in range(bits):
        class_prob.append(float(mycount[b])/qc_shots)
    
    counts = fire_ibmq(p_single_circ_2, qc_shots, Simulation=simulation, quiet=quiet)
    (mycount, bits) = analyze(counts)
    for b in range(bits):
        class_prob.append(float(mycount[b])/qc_shots)

    result = abs((class_prob.index(max(class_prob)) - target[0]).numpy()) # 0 if correct, 1 if not

    p_prob.append(class_prob)
    p_pred.append(result)

    # print(result==0, class_prob[0])
    
    itr += 1
    IPython.display.clear_output(wait=True)
    print("Iterantion", itr, "out of", min(max_itr, len(test_loader))
    if itr >= max_itr:
        break

if saving:
    t = time.localtime()
    dir = "./results/dual_"+time.strftime('%m-%d_%H-%M', t)

    if not os.path.exists(dir):
        os.makedirs(dir)

    df_pred = pd.DataFrame(p_pred)
    df_pred.to_csv(dir+"/predictions.csv", index=False)

    df_prob = pd.DataFrame(p_prob, columns=["p[0]", "p[1]"])
    df_prob.to_csv(dir+"/probilities.csv", index=False)

iterantion 582 out of 2000


In [4]:
data_path = "results\\dual_07-29_05-28"

df_pred = pd.read_csv(data_path + '\\predictions.csv')
p_pred = df_pred.iloc[:, 0].to_numpy()

df_prob = pd.read_csv(data_path + "\\probilities.csv")
p_prob = df_prob.iloc[:, 0:2].to_numpy()

p_acc = 1 - p_pred.sum()/p_pred.shape[0]
print(p_acc)

0.7921747967479675


In [17]:
max_itr = 99999
min(max_itr, len(test_loader))

1968